In [1]:
import matplotlib.pyplot as plt
from datetime import datetime
import folium
import numpy as np
import matplotlib as mpl
from folium.plugins import MarkerCluster
import json
import fiona 
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.ticker as ticker
import pandas as pd

In [2]:
#import WAZE data
df=pd.read_csv('MORA_alerts.csv', encoding="ISO-8859-1")

C:\Users\Meredith.Raymer\Anaconda3\envs\geo-env\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.alert_type.value_counts()

WEATHERHAZARD    520247
ROAD_CLOSED       83210
JAM               42448
ACCIDENT          11490
Name: alert_type, dtype: int64

In [5]:
alerts = df.loc[df.alert_type != 'ROAD_CLOSED']
alerts.alert_type.value_counts()

WEATHERHAZARD    520247
JAM               42448
ACCIDENT          11490
Name: alert_type, dtype: int64

In [6]:
alerts.sub_type.value_counts()

HAZARD_ON_SHOULDER_CAR_STOPPED        412580
HAZARD_ON_ROAD_CONSTRUCTION            28431
HAZARD_ON_ROAD_OBJECT                  20494
JAM_MODERATE_TRAFFIC                   15963
HAZARD_ON_ROAD_CAR_STOPPED             15164
HAZARD_WEATHER_FOG                     13158
JAM_HEAVY_TRAFFIC                      12783
JAM_STAND_STILL_TRAFFIC                 9934
HAZARD_ON_ROAD_POT_HOLE                 5550
HAZARD_ON_ROAD_ROAD_KILL                4344
ACCIDENT_MAJOR                          3634
ACCIDENT_MINOR                          3067
HAZARD_ON_SHOULDER_ANIMALS              2902
HAZARD_ON_SHOULDER                      1945
HAZARD_ON_SHOULDER_MISSING_SIGN         1436
HAZARD_ON_ROAD_ICE                      1415
HAZARD_ON_ROAD                           559
HAZARD_WEATHER                           273
HAZARD_ON_ROAD_TRAFFIC_LIGHT_FAULT       152
HAZARD_WEATHER_FLOOD                     128
HAZARD_WEATHER_HEAVY_SNOW                 95
HAZARD_WEATHER_HAIL                       91
HAZARD_ON_

In [8]:
#converting WAZE data frame into GeoDataFrame with geometry point 
alertPoints = gpd.GeoDataFrame(alerts, geometry=gpd.points_from_xy(alerts.location_lon, alerts.location_lat),crs={'init' :'epsg:4326'})
alertPoints=alertPoints.to_crs(epsg=4326)

In [23]:
#opening park shapefile
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
park = gpd.read_file('ParkOutline.kml', driver='KML')

In [24]:
#clipping WAZE points to those within the park
parkAlerts=gpd.clip(alertPoints,park)

In [26]:
#opening region shapefile
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
region = gpd.read_file('MORA Region.kml', driver='KML')

In [27]:
#clipping WAZE points to those within the region (includes the park)
regionAlerts=gpd.clip(alertPoints,region)

In [28]:
regionAlerts.sub_type.value_counts()

HAZARD_ON_SHOULDER_CAR_STOPPED        35779
JAM_MODERATE_TRAFFIC                  10087
HAZARD_ON_ROAD_CONSTRUCTION            6028
JAM_HEAVY_TRAFFIC                      2938
HAZARD_ON_ROAD_POT_HOLE                2653
JAM_STAND_STILL_TRAFFIC                2483
HAZARD_ON_ROAD_CAR_STOPPED             2296
HAZARD_ON_ROAD_OBJECT                  2099
HAZARD_WEATHER_FOG                      903
ACCIDENT_MAJOR                          524
ACCIDENT_MINOR                          427
HAZARD_ON_ROAD_ROAD_KILL                383
HAZARD_ON_SHOULDER_ANIMALS              224
HAZARD_ON_ROAD_ICE                      202
HAZARD_ON_SHOULDER                      156
HAZARD_ON_SHOULDER_MISSING_SIGN          94
HAZARD_ON_ROAD                           45
HAZARD_WEATHER_FLOOD                     24
HAZARD_WEATHER_HEAVY_SNOW                22
HAZARD_WEATHER                           21
HAZARD_ON_ROAD_TRAFFIC_LIGHT_FAULT       16
HAZARD_WEATHER_HAIL                       4
Name: sub_type, dtype: int64

In [25]:
parkAlerts.sub_type.value_counts()

JAM_STAND_STILL_TRAFFIC           83
HAZARD_ON_ROAD_CONSTRUCTION       12
JAM_HEAVY_TRAFFIC                 10
HAZARD_ON_SHOULDER_CAR_STOPPED     4
JAM_MODERATE_TRAFFIC               2
HAZARD_ON_ROAD_CAR_STOPPED         1
ACCIDENT_MINOR                     1
HAZARD_ON_SHOULDER_ANIMALS         1
Name: sub_type, dtype: int64

In [29]:
#grouping park alerts by subtype
parkSubType=parkAlerts.groupby(['sub_type']).count()
parkSubType=parkSubType.reset_index()
parkSubType=parkSubType[['sub_type','alert_char_crc']].sort_values(by='sub_type')
parkSubType.columns = ['Alert Sub-Type', 'Number of In-Park Alerts']
parkSubType

,Alert Sub-Type,Number of In-Park Alerts
0,ACCIDENT_MINOR,1
1,HAZARD_ON_ROAD_CAR_STOPPED,1
2,HAZARD_ON_ROAD_CONSTRUCTION,12
3,HAZARD_ON_SHOULDER_ANIMALS,1
4,HAZARD_ON_SHOULDER_CAR_STOPPED,4
5,JAM_HEAVY_TRAFFIC,10
6,JAM_MODERATE_TRAFFIC,2
7,JAM_STAND_STILL_TRAFFIC,83


In [30]:
#grouping park alerts by subtype
regionSubType=regionAlerts.groupby(['sub_type']).count()
regionSubType=regionSubType.reset_index()
regionSubType=regionSubType[['sub_type','alert_char_crc']].sort_values(by='sub_type')
regionSubType.columns = ['Alert Sub-Type', 'Number of Region Alerts']
regionSubType

,Alert Sub-Type,Number of Region Alerts
0,ACCIDENT_MAJOR,524
1,ACCIDENT_MINOR,427
2,HAZARD_ON_ROAD,45
3,HAZARD_ON_ROAD_CAR_STOPPED,2296
4,HAZARD_ON_ROAD_CONSTRUCTION,6028
5,HAZARD_ON_ROAD_ICE,202
6,HAZARD_ON_ROAD_OBJECT,2099
7,HAZARD_ON_ROAD_POT_HOLE,2653
8,HAZARD_ON_ROAD_ROAD_KILL,383
9,HAZARD_ON_ROAD_TRAFFIC_LIGHT_FAULT,16


In [32]:
regionSubType['Number of Region Alerts']=regionSubType['Number of Region Alerts']-parkSubType['Number of In-Park Alerts']

In [35]:
TotalR = regionSubType['Number of Region Alerts'].sum()
TotalR

14046.0

In [36]:
TotalP=parkSubType[ 'Number of In-Park Alerts'].sum()
TotalP

114

In [50]:
subTypes = parkSubType.merge(regionSubType, on='Alert Sub-Type' , how='outer')
subTypes

,Alert Sub-Type,Number of In-Park Alerts_x,Number of Region Alerts,Number of In-Park Alerts_y
0,ACCIDENT_MINOR,1.0,425.0,1.0
1,HAZARD_ON_ROAD_CAR_STOPPED,1.0,2294.0,1.0
2,HAZARD_ON_ROAD_CONSTRUCTION,12.0,6020.0,4.0
3,HAZARD_ON_SHOULDER_ANIMALS,1.0,NaN,NaN
4,HAZARD_ON_SHOULDER_CAR_STOPPED,4.0,NaN,NaN
5,JAM_HEAVY_TRAFFIC,10.0,NaN,NaN
6,JAM_MODERATE_TRAFFIC,2.0,NaN,NaN
7,JAM_STAND_STILL_TRAFFIC,83.0,NaN,NaN
8,ACCIDENT_MAJOR,NaN,522.0,1.0
9,HAZARD_ON_ROAD,NaN,21.0,12.0
